In [1]:
import pandas as pd
import spacy
import pickle
import random
pd.set_option("display.max_columns", None)
from collections import Counter
import re
import google_conf
import os
import json

from nltk.corpus import stopwords

In [2]:
metadata_df = pd.read_json("../data/metadata_df.json")

In [16]:
ids = metadata_df["id_kase"].tolist()

In [3]:
mops_data = google_conf.setup(sheet_url="https://docs.google.com/spreadsheets/d/1VbCIAJssHKV9hlRTwzVFfm40CGnHesq53KXjv2qy4OM/edit?usp=sharing", service_account_path="../../ServiceAccountsKey.json")

In [4]:
nlp_spacy = spacy.load('en_core_web_lg') # python -m spacy download en_core_web_lg

In [5]:
tags = ["NOUN", "ADJ", "VERB", "PROPN"]

In [6]:
stops = set(stopwords.words('english'))

In [7]:
def clean_filter(string):
    string = [re.sub("[,.;:!?@#$~^&*{}}°\"\’\[\]\(\)\_\-\d+]+", "", w) for w in string.split()]
    string = " ".join([w[0] + w[1:].lower() for w in string if ((w not in stops) & (len(w)>1))])
    return string
clean_filter("the; L wouldn't _Word1... \"God's PRESS")

"Word God's Press"

In [8]:
def filter_dict(mydict):
    mycounter = Counter()
    for item in mydict.items():#, key=lambda kv: kv[1], reverse=True))
        string = clean_filter(item[0])
        string = " ".join([w for w in string.split() if w not in stops])
        if (len(string) > 1) & (len(string.split()) > 1):
            mycounter.update({string : item[1]})
    return mycounter

In [15]:
id = 0
sourcepath = "../data/large_files/trigrams_raw/"
ngram_dict = json.load(open(sourcepath +  "trigrams_{}.json".format(str(id)), "r"))
len(ngram_dict)

7455

In [10]:
ngram_dict

{'framework of a': 1,
 'illuminating one another': 1,
 'Reunion of the': 2,
 'all parts of': 1,
 'in religious matters.': 1,
 'way.’ 22 As': 1,
 'another. Most bases': 1,
 'REVIEW forces. ‘Certainly': 1,
 '(to use a': 1,
 'even force, such': 1,
 'in the Consultation': 1,
 'the divine hope': 1,
 'leaves unresolved. I': 1,
 'the healing of': 1,
 'social service. But': 1,
 'which men may': 1,
 'churches. In spite': 1,
 '‘The Critics and': 1,
 'context, and as': 1,
 'that is inherent': 1,
 'range from the': 1,
 'the Church, the': 1,
 'insufficient, attention paid': 1,
 'stimulated the efforts': 1,
 'flow and be': 1,
 'for racial equality,': 1,
 'to be faced,': 1,
 'with one another)': 1,
 'which the reconciliation': 1,
 'as the structures': 1,
 'has preserved into': 1,
 'own? And if': 1,
 'of the issues': 1,
 'far we have': 1,
 'and joint action': 1,
 'and His gift': 1,
 'involved in such': 1,
 'to articulate this': 1,
 'to be re-interpreted': 1,
 'that, in spite': 1,
 'unity in one': 1,
 

In [11]:
%%time
ngram_dict_filtered = filter_dict(ngram_dict)
len(ngram_dict_filtered)

CPU times: user 24.1 ms, sys: 851 µs, total: 24.9 ms
Wall time: 25.5 ms


3465

In [12]:
ngram_dict_filtered

Counter({'illuminating one another': 1,
         'religious matters': 1,
         'way As': 1,
         'another Most bases': 1,
         'Review forces ‘certainly': 1,
         'even force': 1,
         'divine hope': 2,
         'leaves unresolved': 2,
         'social service But': 1,
         'men may': 1,
         'churches In spite': 1,
         '‘the Critics': 1,
         'insufficient attention paid': 1,
         'stimulated efforts': 1,
         'racial equality': 2,
         'one another': 10,
         'joint action': 5,
         'His gift': 2,
         'unity one': 2,
         'mission common': 1,
         'one Gospel breaking': 1,
         'Christian fellowship': 1,
         'community life This': 1,
         'Lukas Vxscher': 1,
         'United churches': 1,
         'biblical injunction': 2,
         'important community': 1,
         'degree order': 1,
         'Press Movement Lutheran': 1,
         'worldwide fellowship': 2,
         'community established': 1,
        

In [14]:
destpath = "../data/large_files/trigrams_filtered/"
os.makedirs(destpath)

In [27]:
with open(destpath + "id_{}.json".format(str(id)), "w") as f:
    json.dump(ngram_dict_filtered, f)

In [133]:
#json.load(open(destpath + "id_1.json"))

{'The requirements': 2,
 'form legends': 1,
 'central theme': 1,
 'given doctrine': 2,
 'opponent alike They': 1,
 'outer foreign authority': 1,
 'com parison': 2,
 'Jesus return': 1,
 'Ibid p': 8,
 'glory establish': 1,
 'stands agreement': 1,
 'righteousness Romans': 2,
 'question existence': 2,
 'thought within': 2,
 'p f': 9,
 'spirit Paul': 1,
 'The apostle Paul': 1,
 'become blessed': 2,
 'situation Ibid': 1,
 'sin order': 1,
 'collect interpret': 1,
 'three figures': 1,
 'clear even': 1,
 'means teaching': 2,
 'less astonishment': 2,
 'existence three': 1,
 'Church Paul': 1,
 'instead solving': 1,
 'Good works': 1,
 'resignare ad infernum': 1,
 'notorious little word': 1,
 'fall sin': 1,
 'Luther knew': 1,
 'The present treatise': 1,
 'OF GRACE II': 1,
 'types At': 1,
 'cultic practices': 2,
 'trust Similarly one': 1,
 'symbolic form': 2,
 'world Kingdom': 1,
 'civil use': 2,
 'rans doctrine': 1,
 'reason proofs': 1,
 'The changes': 1,
 'Writings Shin': 2,
 'Consequently one mig

In [17]:
%%time
data_ngrams = Counter()
sourcepath = "../data/large_files/trigrams_raw/"
for id in ids:
    ngram_dict = json.load(open(sourcepath +  "trigrams_{}.json".format(str(id)), "r"))
    d_cleaned = filter_dict(ngram_dict)
    with open(destpath + "filtered_id_{}.json".format(str(id)), "w") as f:
        json.dump(d_cleaned, f)
    data_ngrams.update(d_cleaned)

CPU times: user 13min 53s, sys: 22.3 s, total: 14min 15s
Wall time: 15min


In [18]:
len(data_ngrams)

45296539

In [19]:
%%time
data_trigrams_sorted = sorted(data_ngrams.items(), key=lambda kv: kv[1], reverse=True)
len([el for el in data_trigrams_sorted if el[1] > 10])

CPU times: user 6.15 s, sys: 2.57 s, total: 8.72 s
Wall time: 9.93 s


1325945

In [20]:
to_lemmatize = [el for el in data_trigrams_sorted if el[1] > 5]
len(to_lemmatize)

2330940

In [23]:
# use spacy to generate "Doc" object for ngram
# (use carefully, takes some time)
tags = ["NOUN", "ADJ", "VERB", "PROPN"]

def get_lemma_filtered(string):
    #lemmata = " ".join([t.lemma for t in nlp_stanza(string).sentences[0].words if t.pos in tags])
    lemmata = " ".join([t.lemma_ for t in nlp_spacy(string) if t.pos_ in tags])
    return lemmata

In [24]:
monitor = [n for n in range(0, )]

In [25]:
%%time
monitor = [n for n in range(0, len(to_lemmatize), 2000)]
trigrams_lemmata_dict= {}
for n, tup in enumerate(to_lemmatize):
    lemmata = get_lemma_filtered(tup[0])
    if n in monitor:
        print(n)
    if len(lemmata.split()) > 1:
        trigrams_lemmata_dict[tup[0]] = {"lemmata" : lemmata, "count" : tup[1]}

0
2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000
24000
26000
28000
30000
32000
34000
36000
38000
40000
42000
44000
46000
48000
50000
52000
54000
56000
58000
60000
62000
64000
66000
68000
70000
72000
74000
76000
78000
80000
82000
84000
86000
88000
90000
92000
94000
96000
98000
100000
102000
104000
106000
108000
110000
112000
114000
116000
118000
120000
122000
124000
126000
128000
130000
132000
134000
136000
138000
140000
142000
144000
146000
148000
150000
152000
154000
156000
158000
160000
162000
164000
166000
168000
170000
172000
174000
176000
178000
180000
182000
184000
186000
188000
190000
192000
194000
196000
198000
200000
202000
204000
206000
208000
210000
212000
214000
216000
218000
220000
222000
224000
226000
228000
230000
232000
234000
236000
238000
240000
242000
244000
246000
248000
250000
252000
254000
256000
258000
260000
262000
264000
266000
268000
270000
272000
274000
276000
278000
280000
282000
284000
286000
288000
290000
292000
294000
296000
298000
300000
3

In [26]:
with open("../data/large_files/trigrams_lemmata_dict.pickle", "wb") as f:
    pickle.dump(trigrams_lemmata_dict, f)

In [9]:
#trigrams_lemmata_dict = pickle.load(open("../data/large_files/trigrams_lemmata_dict.pickle", "rb"))
#len(trigrams_lemmata_dict)

1246103

In [27]:
%%time
# developing application to individual documents
sourcepath = "../data/large_files/trigrams_filtered/"
id = 0
with open(sourcepath + "filtered_id_{}.json".format(str(id)), "r") as f:
    d_cleaned = json.load(f)
counter = Counter()
for item in d_cleaned.items():
    try:
        counter.update({trigrams_lemmata_dict[item[0]]["lemmata"] : item[1]})
    except:
        pass

CPU times: user 3.1 ms, sys: 4.46 ms, total: 7.56 ms
Wall time: 8.07 ms


In [28]:
trigrams_list = []
for tup in counter.items():
    trigrams_list.extend([tup[0]]*tup[1])

In [29]:
trigrams_list[:100]

['religious matter',
 'divine hope',
 'divine hope',
 'leave unresolved',
 'leave unresolved',
 'racial equality',
 'racial equality',
 'joint action',
 'joint action',
 'joint action',
 'joint action',
 'joint action',
 'christian fellowship',
 'biblical injunction',
 'biblical injunction',
 'important community',
 'degree order',
 'worldwide fellowship',
 'worldwide fellowship',
 'community establish',
 'new appreciation',
 'new appreciation',
 'Council church',
 'Council church',
 'local fellowship',
 'local fellowship',
 'establish maintain',
 'op cit',
 'op cit',
 'op cit',
 'op cit',
 'op cit',
 'op cit',
 'op cit',
 'op cit',
 'op cit',
 'op cit',
 'op cit',
 'Wcc Commission',
 'Wcc Commission',
 'use several',
 'nego tiating',
 'nego tiating',
 'church provide',
 'church provide',
 'church provide',
 'Association Press',
 'Association Press',
 'loss identity',
 'particular concept',
 'con versation',
 'form life',
 'doctrinal agreement',
 'genuine renewal',
 'genuine renewal',


In [30]:
sourcepath = destpath
destpath = "../data/large_files/trigrams_lemmatized/"
try:
    os.makedirs(destpath)
except:
    pass

In [33]:
%%time
for id in ids:
    with open(sourcepath + "filtered_id_{}.json".format(str(id)), "r") as f:
        d_cleaned = json.load(f)
    counter = Counter()
    for item in d_cleaned.items():
        try:
            counter.update({trigrams_lemmata_dict[item[0]]["lemmata"] : item[1]})
        except:
            pass
    trigrams_list = []
    for tup in counter.items():
        trigrams_list.extend([tup[0]]*tup[1])
    with open(destpath + "trigrams_lemmata_id_{}.txt".format(str(id)), "w") as f:
        f.writelines(trigrams_list)

CPU times: user 2min 4s, sys: 20.4 s, total: 2min 25s
Wall time: 2min 50s


# Backup

# Most frequent unigrams to nlp docs

In [108]:
# load spacy english nlp model & english stopwords...
#stop_words = nlp_spacy.Defaults.stop_words

In [109]:
# check how raw unigrams look like
data_unigrams_sorted = sorted(data_unigrams.items(), key=lambda kv: kv[1], reverse=True)
data_unigrams_sorted[:1000]

[('the', 64204),
 ('of', 43727),
 ('and', 29112),
 ('to', 24051),
 ('in', 23227),
 ('is', 11614),
 ('that', 11283),
 ('as', 8204),
 ('The', 6860),
 ('for', 6850),
 ('his', 6283),
 ('with', 6121),
 ('was', 5995),
 ('he', 5314),
 ('by', 5164),
 ('not', 5095),
 ('on', 4746),
 ('from', 4525),
 ('it', 4500),
 ('this', 4388),
 ('be', 4355),
 ('or', 3657),
 ('which', 3568),
 ('are', 3401),
 ('an', 3185),
 ('their', 3085),
 ('who', 3080),
 ('God', 3032),
 ('have', 2951),
 ('but', 2774),
 ('at', 2649),
 ('Paul', 2601),
 ('were', 2543),
 ('they', 2531),
 ('In', 2524),
 ('had', 2435),
 ('one', 2336),
 ('all', 2258),
 ('has', 2093),
 ('him', 1966),
 ('we', 1966),
 ('Christ', 1886),
 ('also', 1854),
 ('its', 1685),
 ('them', 1600),
 ('Christian', 1594),
 ('more', 1579),
 ('Jesus', 1546),
 ('would', 1496),
 ('what', 1489),
 ('you', 1474),
 ('This', 1472),
 ('been', 1441),
 ('will', 1439),
 ('other', 1423),
 ('into', 1356),
 ('only', 1342),
 ('It', 1324),
 ('so', 1322),
 ('no', 1299),
 ('about', 1293

In [110]:
data_unigrams_sorted[:1000]

[('the', 64204),
 ('of', 43727),
 ('and', 29112),
 ('to', 24051),
 ('in', 23227),
 ('is', 11614),
 ('that', 11283),
 ('as', 8204),
 ('The', 6860),
 ('for', 6850),
 ('his', 6283),
 ('with', 6121),
 ('was', 5995),
 ('he', 5314),
 ('by', 5164),
 ('not', 5095),
 ('on', 4746),
 ('from', 4525),
 ('it', 4500),
 ('this', 4388),
 ('be', 4355),
 ('or', 3657),
 ('which', 3568),
 ('are', 3401),
 ('an', 3185),
 ('their', 3085),
 ('who', 3080),
 ('God', 3032),
 ('have', 2951),
 ('but', 2774),
 ('at', 2649),
 ('Paul', 2601),
 ('were', 2543),
 ('they', 2531),
 ('In', 2524),
 ('had', 2435),
 ('one', 2336),
 ('all', 2258),
 ('has', 2093),
 ('him', 1966),
 ('we', 1966),
 ('Christ', 1886),
 ('also', 1854),
 ('its', 1685),
 ('them', 1600),
 ('Christian', 1594),
 ('more', 1579),
 ('Jesus', 1546),
 ('would', 1496),
 ('what', 1489),
 ('you', 1474),
 ('This', 1472),
 ('been', 1441),
 ('will', 1439),
 ('other', 1423),
 ('into', 1356),
 ('only', 1342),
 ('It', 1324),
 ('so', 1322),
 ('no', 1299),
 ('about', 1293

In [111]:
# how many of the 100 most frequent unigrams are actually not stopwords?
#len([el for el in data_unigrams_sorted[:100] if el[0].lower() not in stop_words])

In [113]:
# what is the number of unigrams appearing more than 10 times?
len([el for el in data_unigrams_sorted if el[1] > 10])

7810

In [124]:

# use spacy to generate "Doc" object for ngram
# (use carefully, takes some time)
tags = ["NOUN", "ADJ", "VERB", "PROPN"]
def get_lemma_filtered(string):
    lemmata = " ".join([t.lemma for t in nlp_stanza(string).sentences[0].words if t.pos in tags])
    return lemmata

In [131]:
data_unigrams_lemmata = Counter()
for el in data_unigrams_sorted:
    if el[1] > 10:
        lemmata_str = get_lemma_filtered(el[0])
        if len(lemmata_str) > 1:
            data_unigrams_lemmata.update({lemmata_str : el[1]})

In [ ]:
data_unigrams = merge_data_from_ids(all, unigramCount_dict)
len(data_unigrams)

In [ ]:
# check how raw unigrams look like
data_unigrams_sorted = sorted(data_unigrams.items(), key=lambda kv: kv[1], reverse=True)
data_unigrams_sorted[:1000]


In [ ]:
[(el[0], {"doc" : get_lemma_filtered(el[0]), "count" : el[1]}) for el in data_unigrams_sorted if el[1] > 1000]

In [34]:

data_unigrams_sorted_nlp = [(el[0], {"doc" : get_lemma_filtered(el[0]), "count" : el[1]}) for el in data_unigrams_sorted if el[1] > 1000]

#data_unigrams_sorted_nlp = [(el[0], {"doc" : nlp_stanza(el[0]), "count" : el[1]}) for el in data_unigrams_sorted if el[1] > 1000]

CPU times: user 1min 12s, sys: 7.5 s, total: 1min 20s
Wall time: 1min 21s


In [38]:
data_unigrams_sorted_nlp[501]

('G.',
 {'doc': [
    [
      {
        "id": 1,
        "text": "G.",
        "lemma": "G.",
        "upos": "X",
        "xpos": "ADD",
        "head": 0,
        "deprel": "root",
        "start_char": 0,
        "end_char": 2,
        "ner": "O",
        "multi_ner": [
          "O"
        ]
      }
    ]
  ],
  'count': 1856})

In [15]:
data_unigrams_sorted_nlp[:5]

[('the', {'doc': the, 'count': 3333595}),
 ('of', {'doc': of, 'count': 2261372}),
 ('and', {'doc': and, 'count': 1459413}),
 ('to', {'doc': to, 'count': 1228717}),
 ('in', {'doc': in, 'count': 1151878})]

In [16]:
data_unigrams_nlp_dict = dict(data_unigrams_sorted_nlp)

In [17]:
# save for future usage
with open("../data/large_files/data_unigrams_nlp_dict.pickle", "wb") as f:
    pickle.dump(data_unigrams_nlp_dict, f)

In [18]:
# remove middle-step data to save CPU...
del data_unigrams
del data_unigrams_sorted
del data_unigrams_sorted_nlp

# Trigrams to nlp docs

In [ ]:
trigramCount_dict = pickle.load(open("../data/large_files/trigramCount_dict.pickle", "rb"))

In [ ]:
#data_unigrams = merge_data_from_ids(ids, unigramCount_dict)
#data_bigrams = merge_data_from_ids(ids, bigramCount_dict)
data_trigrams = merge_data_from_ids(ids, trigramCount_dict)

In [ ]:
ids2 = [k for k in list(trigramCount_dict.keys()) if k not in ids]
len(ids2)

In [ ]:
data_trigrams2 = merge_data_from_ids(ids2, trigramCount_dict)

In [ ]:
del trigramCount_dict

In [11]:
data_trigrams_sorted = sorted(data_trigrams.items(), key=lambda kv: kv[1], reverse=True)
len([el for el in data_trigrams_sorted if el[1] > 10])

458444

In [26]:
%%time
data_trigrams_sorted_nlp = [(el[0], {"doc" : nlp(el[0]), "count" : el[1]}) for el in data_trigrams_sorted if el[1] > 10]

CPU times: user 21min 37s, sys: 33.8 s, total: 22min 11s
Wall time: 23min 14s


In [16]:
del data_trigrams_sorted

In [27]:
data_trigrams_nlp_dict = dict(data_trigrams_sorted_nlp)

In [28]:
with open("../data/large_files/data_trigrams_nlp_dict.pickle", "wb") as f:
    pickle.dump(data_trigrams_nlp_dict, f)

# Second round of merged trigrams nlp preprocessing

In [12]:
len(data_trigrams2)

46627339

In [42]:
data_trigrams2 = dict(data_trigrams2)

In [13]:
keys1 = [el[0] for el in data_trigrams.items() if el[1] > 10]
len(keys1)

458444

In [22]:
del data_trigrams

In [14]:
keys2 = [el[0] for el in data_trigrams2.items() if el[1] > 10]
len(keys2) #  [el[0] for el in data_trigrams2_filtered]

774574

In [15]:
keys2_filtered = list((set(keys2) ^ set(keys1)) & set(keys2))
len(keys2_filtered)

342693

In [31]:
del nlp

In [32]:
print("hello")

hello


In [39]:
pickle.dump(keys2_filtered, open("../data/large_files/keys2_filtered.pickle", "wb"))
pickle.dump(data_trigrams2, open("../data/large_files/data_trigrams2.pickle", "wb"))

In [40]:
len(data_trigrams2)

46627339

In [43]:
%%time
data_trigrams2_filtered = []
n = 0
for el in keys2_filtered:
    if n in range(0, len(keys2_filtered), 5000):
        print(n)
    n += 1
    data_trigrams2_filtered.append((el, data_trigrams2[el]))

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000
235000
240000
245000
250000
255000
260000
265000
270000
275000
280000
285000
290000
295000
300000
305000
310000
315000
320000
325000
330000
335000
340000
CPU times: user 285 ms, sys: 286 ms, total: 572 ms
Wall time: 839 ms


In [44]:
len(data_trigrams2_filtered)

342693

In [46]:
pickle.dump(data_trigrams2_filtered, open("../data/large_files/data_trigrams2_filtered.pickle", "wb"))

In [47]:
nlp = spacy.load('en_core_web_lg')

In [48]:
%%time
data_trigrams2_nlp_tups = []
n = 0
for el in data_trigrams2_filtered:
    if n in range(0, len(data_trigrams2_filtered), 5000):
        print(n)
    n += 1
    data_trigrams2_nlp_tups.append((el[0], {"doc" : nlp(el[0]), "count" : el[1]}))

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000
235000
240000
245000
250000
255000
260000
265000
270000
275000
280000
285000
290000
295000
300000
305000
310000
315000
320000
325000
330000
335000
340000
CPU times: user 14min 25s, sys: 14.3 s, total: 14min 39s
Wall time: 14min 44s


In [49]:
pickle.dump(dict(data_trigrams2_nlp_tups), open("../data/large_files/data_trigrams2_nlp_dict.pickle", "wb"))